# Stretch Body Guide: Commanding Motions

The Stretch Body package provides a low level Python API to the Stretch RE1 hardware. In this guide, we'll look at using the package to command motions to the robot.

## Setup

Stretch Body is available on PyPi as [hello-robot-stretch-body](https://pypi.org/project/hello-robot-stretch-body/). An accompanying command-line tools package called [hello-robot-stretch-body-tools](https://pypi.org/project/hello-robot-stretch-body-tools/) uses Stretch Body to provide convenient scripts to interact with the robot from the command-line. Both come preinstalled on Stretch RE1, but the following command can be used to ensure you have the latest version.

In Jupyter notebooks, code preceded by `!` are run in the command-line instead of the Python interpreter.

In [ ]:
!python -m pip install -q -U hello-robot-stretch-body
!python -m pip list | grep hello-robot-stretch-body

# The Robot Class

Stretch Body exposes many classes, but we'll focus on the `stretch_body.robot.Robot` class to start. This Python class allows users to command motions to the entire robot. Additionally, it allows the user to access the current state of the robot.

We'll import the `stretch_body.robot` module, as well as the `numpy` and `time` modules for other utility methods.

In [ ]:
import time
import numpy as np
import stretch_body.robot

Now, we'll instantiate an object of robot and call it `r`.

In [ ]:
r = stretch_body.robot.Robot()

The first method we'll look at is called `startup()`. The cell below displays the docstring for `startup()`. As you can see, the method returns a boolean depending on whether or not the class startup procedure succeeded. Only one instance of the robot class can exist at once, so if another instance is running elsewhere, the method returns false.

In [ ]:
r.startup?

Below, we make the call to `startup()`. As part of the startup procedure, this method opens serial ports to the hardware devices, loads the parameters that dictate robot behavior, and launches a few helper threads to poll for status in the background.

In [ ]:
r.startup()

If startup fails unexpectedly, the first thing to check is whether a background process is already running an instance of the robot class. Below we use the `pstree` command to list the tree of background processes, and `grep` to filter for scripts starting with "stretch_" (often the "stretch_xbox_controller_teleop.py" scripts is running in the background). If we see output below, we should use the `pkill` command to [terminate the conflicting process](https://docs.hello-robot.com/troubleshooting_guide/#rpc-transport-errors-stretch-doesnt-respond-to-commands).

In [ ]:
!pstree | grep stretch_

The second method we'll look at is called `stop()`. This method closes connections to all serial ports, releases other resources, and shuts down background threads. We'll wait until the end of the notebook to actually call the method.

In [ ]:
r.stop?

With the startup procedure completed, we must check that the robot is "homed". The relevant method is called `is_homed()` and returns a boolean depending on whether the hardware has been homed.

In [ ]:
r.is_calibrated?

The robot must be homed once every time it has been turned on. The homing procedure allows the robot to determine where its joints are with respect to the limits of motion.

In [ ]:
r.is_calibrated()

The `home()` method puts the Stretch RE1 through the homing procedure. The method is blocking and will not return until all joints on the robot is homed. It returns a boolean depending on whether the robot homes successfully.

In [ ]:
r.home()

The hardware now reports that the robot is homed.

In [ ]:
r.is_calibrated()

Next, we'll send the robot to its "stow" position with the `stow()` method. A custom stow position can be defined for any end-effector attached to the robot, however, the default position retracts the Arm and Wrist Yaw, and sends the Lift to 0.2m above the base. `stow()` is blocking as well and returns a boolean depending on whether stowing succeeds.

In [ ]:
r.stow()

The next method we'll look at is called `pretty_print()`. This method prints out the entire state of the robot in a human readable format.

In [ ]:
r.pretty_print()

However, we'd often like to be able to access the robot's state programmatically. The `get_status()` method returns a dictionary with a snapshot of the current state of the robot.

In [ ]:
r.get_status()

Since we stowed the robot before, we expect the Lift to be at 0.2 meters. We can verify it like this:

In [ ]:
stow_status = r.get_status()
lift_pos_m = stow_status['lift']['pos']
is_lift_close = np.isclose(lift_pos_m, 0.2, atol=1e-3)

print('Lift position {0}m is near 0.2m? {1}'.format(lift_pos_m, is_lift_close))

Note that the status dictionary is a snapshot of the state of the robot when the method was called. If we change the state of the robot, the dictionary will be out of date.

We'll look at one last method from the robot class called `push_command()`. This method takes commands queued at from the Python API and pushes it out to the hardware drivers.

In [ ]:
r.push_command?

# Joint Classes

Next, let's look at the classes within Stretch Body that we can send motion commands to.

 * `stretch_body.lift.Lift`: maintains the lift, a vertical prismatic joint
 * `stretch_body.arm.Arm`: maintains the arm, a telescoping prismatic joint
 * `stretch_body.base.Base`: maintains the mobile base, containing left and right motors in a diff drive configuration
 * `stretch_body.head.Head`: maintains the head, containing tilt and pan revolute joints
 * `stretch_body.end_of_arm.EndOfArm`: maintains the end-effector, typically a wrist yaw joint and a compliant gripper

The robot class has an instance of each one of these joints, however, we could have instantiated a single joint to interact with just it. For example, we could create a lift object using `l = stretch_body.lift.Lift()`. Then, most of the methods we covered with the robot class can be used with the lift class, including `l.startup()`, `l.stop()`, `l.home()`, and `l.pretty_print()`. The same applies to the other joint classes too.

The robot class makes an instance of each joint on the robot and handles startup, homing, and stopping automatically. Therefore, to interact with the lift joint, we can access its class from `r.lift`.

In [ ]:
r.lift

Let's use robot's joint classes to determine the range of motion of each joint on the robot. Each joint class has an attribute called `params`, which is a dictionary of parameters dictating how the class behavies. We can find the joint range in these dictionaries.

In [ ]:
lift_range_m = r.lift.params['range_m']
arm_range_m = r.arm.params['range_m']
wrist_yaw_range_rad = r.end_of_arm.motors['wrist_yaw'].soft_motion_limits
stretch_gripper_range_rad = r.end_of_arm.motors['stretch_gripper'].soft_motion_limits
head_tilt_range_rad = r.head.motors['head_tilt'].soft_motion_limits
head_pan_range_rad = r.head.motors['head_pan'].soft_motion_limits

is_lift_within_range = lambda pos: pos >= lift_range_m[0] and pos <= lift_range_m[1]
is_arm_within_range = lambda pos: pos >= arm_range_m[0] and pos <= arm_range_m[1]
is_wrist_yaw_within_range = lambda pos: pos >= wrist_yaw_range_rad[0] and pos <= wrist_yaw_range_rad[1]
is_stretch_gripper_within_range = lambda pos: pos >= stretch_gripper_range_rad[0] and pos <= stretch_gripper_range_rad[1]
is_head_tilt_within_range = lambda pos: pos >= head_tilt_range_rad[0] and pos <= head_tilt_range_rad[1]
is_head_pan_within_range = lambda pos: pos >= head_pan_range_rad[0] and pos <= head_pan_range_rad[1]

print('Lift range is {0} meters'.format(lift_range_m))
print('Arm range is {0} meters'.format(arm_range_m))
print('Mobile base has no joint limits')
print('End effector wrist_yaw range is {0} radians'.format(wrist_yaw_range_rad))
print('End effector gripper range is {0} radians'.format(stretch_gripper_range_rad))
print('Head tilt range is {0} radians'.format(head_tilt_range_rad))
print('Head pan range is {0} radians'.format(head_pan_range_rad))

The joint classes make two methods available to allow users to issue position commands. The `move_to()` and `move_by()` methods move a joint to a specific position value and by a position delta respectively.

In [ ]:
r.lift.move_to?

We combine `move_to()` and `move_by()` commands with `push_command()` to trigger execution of the motion command. In the cell below, we command the robot to move to 0.3 meters above the base. Since the robot had been in the stow position, we expect it to move upwards by 10 centimeters.

In [ ]:
r.lift.move_to(0.3)
r.push_command() # pushes command to the hardware

Let's confirm that lift is at 0.3m and within the valid range of the lift (typically 0.0 to 1.1 meters).

In [ ]:
moveto_status = r.get_status()
lift_pos_m = moveto_status['lift']['pos']
is_lift_close = np.isclose(lift_pos_m, 0.3, atol=1e-3)
lift_within_range = is_lift_within_range(lift_pos_m)

print('Lift position {0}m is near 0.3m and within lift range of {1}? {2}'.format(lift_pos_m, lift_range_m, is_lift_close and lift_within_range))

Note that `push_command()` doesn't block until the motion is completed. This allows your code to monitor the robot state and even send overriding commands while the robot is executing a motion. Below, we send the lift to 0.2m again and monitor the lift position until it reaches the desired goal. We loop until the lift has stopped moving by checking the "is_moving_filtered" flag.

In [ ]:
r.lift.move_by(-0.1)
r.push_command()

time.sleep(0.4) # give the lift some time to start moving
while r.get_status()['lift']['motor']['is_moving_filtered']:
    time.sleep(0.2)
    print(r.get_status()['lift']['pos'])

While `r.lift`, `r.arm`, `r.end_of_arm`, and `r.head` have the `move_to()` and `move_by()` methods, the mobile base `r.base` has slightly different methods. Because the mobile base can translate and rotate, the mobile base has `translate_by()` and `rotate_by()`.

In [ ]:
r.base.rotate_by?

The cell below commands the robot to rotate 45 degrees. A commented command can send the robot in a full circle. Be wary of cables attached to the base before uncommenting this command.

In [ ]:
r.base.rotate_by(np.pi / 4)
# r.base.rotate_by(np.pi * 2) # rotates a full circle, check no cables will get caught
r.push_command()

The `push_command()` method triggers synchronous motion of all issued commands. This is often important in robotics because tasks like grasping required coordinated execution of all joints together. Below, we issue a whole body coordinated command.

In [ ]:
r.base.rotate_by(-np.pi / 4)
r.lift.move_by(0.3)
r.arm.move_to(0.2)
r.end_of_arm.move_to("stretch_gripper", 0.0)
r.end_of_arm.move_to("wrist_yaw", 0.0)
r.head.move_to("head_tilt", -np.pi / 4)
r.head.move_to("head_pan", -np.pi)
r.push_command()

In [ ]:
r.stow()

# Wrapping Up

In this notebook, we've covered:

 * Classes and methods available in Stretch Body 
 * Fetching robot state from the `stretch_body.robot.Robot` class
 * Commanding motions to the various joints on the robot.

For more information on Stretch Body API, take a look at the [API Documentation](https://docs.hello-robot.com/stretch_body_guide/). To reports bugs or contribute to the library, visit the [Stretch Body Github repo](https://github.com/hello-robot/stretch_body/) where development on the library happens. Also, feel free to join our community on the [forum](https://forum.hello-robot.com/) and learn about research/projects happening with Stretch.

In [ ]:
r.stop()